***Classifying The Severity Of Alzheimer's Attack Using Convolutional Neural Network And MRI Model***

**Importing libraries**

In [ ]:
import os
import numpy 
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
#for loading a manipulating images
import PIL
#tf.distribute.Strategy is a TensorFlow API to distribute training across multiple GPUs
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)
    
print(tf.__version__)

Number of replicas: 1
2.8.0


In [ ]:
!unzip /content/Alzheimer_sDataset.zip

In [ ]:
class_names = ['MildDementia', 'ModerateDementia', 'NonDementia', 'VeryMildDementia']
train = class_names
test = class_names

NUM_CLASSES = len(class_names)

In [ ]:
#number of passes of the entire training dataset 
AUTOTUNE = tf.data.experimental.AUTOTUNE
IMAGE_SIZE = [176, 208]
# training examples used in 1 iteration
BATCH_SIZE = 16 * strategy.num_replicas_in_sync 
EPOCHS = 15

In [ ]:
#We are also specifying a 80:20 split for our training and validation datasets.
train = tf.keras.preprocessing.image_dataset_from_directory(
   'Alzheimer_s Dataset/train',
    validation_split=0.8,
    subset="training",
    seed=1337,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
)

test = tf.keras.preprocessing.image_dataset_from_directory(
    'Alzheimer_s Dataset/train',
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
)

In [ ]:
#visualize the dataset
plt.figure(figsize=(10, 10))
for images, labels in train.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(train.class_names[labels[i]])
    plt.axis("off")

In [ ]:
#One-hot encodings are a way for the model to understand that we're looking at categorial instead of continuous data.
def one_hot_label(image, label):
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

train = train.map(one_hot_label, num_parallel_calls=AUTOTUNE)
test = test.map(one_hot_label, num_parallel_calls=AUTOTUNE)

# **Create the convolutional base**

In [ ]:
def conv_block(filters):
    block = tf.keras.Sequential([
        tf.keras.layers.SeparableConv2D(filters, 3, activation='relu', padding='same'),
        tf.keras.layers.SeparableConv2D(filters, 3, activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D()
    ])
    
    return block

In [ ]:
def dense_block(units, dropout_rate):
    block = tf.keras.Sequential([
        tf.keras.layers.Dense(units, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(dropout_rate)
    ])
    
    return block

In [ ]:
def build_model():
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(*IMAGE_SIZE, 3)),
        
        tf.keras.layers.Conv2D(16, 3, activation='relu', padding='same'),
        tf.keras.layers.Conv2D(16, 3, activation='relu', padding='same'),
        tf.keras.layers.MaxPool2D(),
        
        conv_block(32),
        conv_block(64),
        
        conv_block(128),
        tf.keras.layers.Dropout(0.2),
        
        conv_block(256),
        tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.Flatten(),
        dense_block(512, 0.7),
        dense_block(128, 0.5),
        dense_block(64, 0.3),
        
        tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    
    return model

In [ ]:
with strategy.scope():
    model = build_model()

    METRICS = [tf.keras.metrics.AUC(name='auc')]
    
    model.compile(
        optimizer='adam',
        loss=tf.losses.CategoricalCrossentropy(),
        metrics=METRICS
    )

In [1]:
model.summary()

NameError: ignored

**To more efficiently train our model. We will be using callbacks to adjust our learning rate and to stop our model once it converges.**

In [ ]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.01, 20)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("alzheimer_model.h5",
                                                    save_best_only=True)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10,
                                                     restore_best_weights=True)

In [ ]:
history = model.fit(
    train,
    validation_data= test,
    callbacks=[checkpoint_cb, early_stopping_cb, lr_scheduler],
    epochs=EPOCHS
)

In [ ]:
_ = model.evaluate(test)